# bikeレンタルの特徴量の解釈
https://christophm.github.io/interpretable-ml-book/limo.html

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from pandas import Series, DataFrame
import pandas as pd
%matplotlib inline
os.getcwd()

In [ ]:
data_dir = '../../data/bike'
day_file = 'day.csv'
hour_file = 'hour.csv'

## データの意味

- cnt:casualとregisteredのユーザーの数。回帰のターゲット
- season:spring, summer, fall or winter.
- holiday
- year: 2011 or 2012.
- dateday:2011/01/01からの日数。この特料量は時間経過によるトレンドを導くのに使用する。
- working day or weekend.
- 天気:
    - clear, few clouds, partly cloudy, cloudy
    - mist + clouds, mist + broken clouds, mist + few clouds, mist
    - light snow, light rain + thunderstorm + scattered clouds, light rain + scattered clouds
    - heavy rain + ice pallets + thunderstorm + mist, snow + mist
- 温度：Temperature in degrees Celsius.41で割り、正規化されている。
- atemp：体感温度。50で割り、正規化されている。
- hum:湿度(0 to 100). 100で割り正規化されている。
- Wind speed: km/hour.　67で割り、正規化されている。


In [ ]:
df_day = pd.read_csv(os.path.join(data_dir, day_file))
df_day


In [ ]:
# データを取り出す
Y = df_day['cnt']
X = df_day['hum'] * 100

In [ ]:
# データの確認
fig = plt.figure()

ax = fig.add_subplot(1,1,1)

ax.scatter(X,Y)

ax.set_title('first scatter plot')
ax.set_xlabel('x')
ax.set_ylabel('y')

fig.show()

In [ ]:
# https://qiita.com/yshi12/items/9f3f8ae69588da9f018f
#X = np.array([[value, 1] for value in X])
#Y = np.array([[value, 1] for value in Y])

X = np.c_[X]
Y = np.c_[Y]

In [ ]:
# 線形回帰
from sklearn import linear_model
model = linear_model.LinearRegression()
#X = X.reshape(1, -1)
#Y = Y.reshape(1, -1)

model.fit(X, Y)

In [ ]:
model.predict([[0.5]])

In [ ]:
model.coef_